# Module 10 - Programming Assignment

## Directions

1. Change the name of this file to be your JHED id as in `jsmith299.ipynb`. Because sure you use your JHED ID (it's made out of your name and not your student id which is just letters and numbers).
2. Make sure the notebook you submit is cleanly and fully executed. I do not grade unexecuted notebooks.
3. Submit your notebook back in Blackboard where you downloaded this file.

*Provide the output **exactly** as requested*

# Forward Planner

## Unify

Use the accompanying `unification.py` file for unification. For this assignment, you're almost certainly going to want to be able to:

1. specify the problem in terms of S-expressions.
2. parse them.
3. work with the parsed versions.

`parse` and `unification` work exactly like the programming assignment for last time.

In [1]:
from unification import parse, unification
from typing import List, Dict, Any, Set, Tuple
from copy import deepcopy

## Forward Planner

In this assigment, you're going to implement a Forward Planner. What does that mean? If you look in your book, you will not find pseudocode for a forward planner. It just says "use state space search" but this is less than helpful and it's a bit more complicated than that. **(but please please do not try to implement STRIPS or GraphPlan...that is wrong).**

At a high level, a forward planner takes the current state of the world $S_0$ and attempts to derive a plan, basically by Depth First Search. We have all the ingredients we said we would need in Module 1: states, actions, a transition function and a goal test. We have a set of predicates that describe a state (and therefore all possible states), we have actions and we have, at least, an implicit transition function: applying an action in a state causes the state to change as described by the add and delete lists.

Let's say we have a drill that's an item, two places such as home and store, and we know that I'm at home and the drill is at the store and I want to go buy a drill (have it be at home). We might represent that as:

<code>
start_state = [
    "(item Saw)",
    "(item Drill)",
    "(place Home)",
    "(place Store)",
    "(place Bank)",
    "(agent Me)",
    "(at Me Home)",
    "(at Saw Store)",
    "(at Drill Store)",
    "(at Money Bank)"
]
</code>

And we have a goal state:

<code>
goal = [
    "(item Saw)",
    "(item Drill)",
    "(place Home)",
    "(place Store)",
    "(place Bank)",
    "(agent Me)",
    "(at Me Home)",
    "(at Drill Me)",
    "(at Saw Store)",
    "(at Money Bank)"
]
</code>

The actions/operators are:

<code>
actions = {
    "drive": {
        "action": "(drive ?agent ?from ?to)",
        "conditions": [
            "(agent ?agent)",
            "(place ?from)",
            "(place ?to)",
            "(at ?agent ?from)"
        ],
        "add": [
            "(at ?agent ?to)"
        ],
        "delete": [
            "(at ?agent ?from)"
        ]
    },
    "buy": {
        "action": "(buy ?purchaser ?seller ?item)",
        "conditions": [
            "(item ?item)",
            "(place ?seller)",
            "(agent ?purchaser)",
            "(at ?item ?seller)",
            "(at ?purchaser ?seller)"
        ],
        "add": [
            "(at ?item ?purchaser)"
        ],
        "delete": [
            "(at ?item ?seller)"
        ]
    }
}
</code>

These will all need to be parsed from s-expressions to the underlying Python representation before you can use them. You might as well do it at the start of your algorithm, once. The order of the conditions is *not* arbitrary. It is much, much better for the unification and backtracking if you have the "type" predicates (item, place, agent) before the more complex ones. Trust me on this.

As for the algorithm itself, there is going to be an *outer* level of search and an *inner* level of search.

The *outer* level of search that is exactly what I describe here: you have a state, you generate successor states by applying actions to the current state, you examine those successor states as we did at the first week of the semester and if one is the goal you stop, if you see a repeat state, you put it on the explored list (you should implement graph search not tree search). What could be simpler?

It turns out the Devil is in the details. There is an *inner* level of search hidden in "you generate successor states by applying actions to the current state". Where?

How do you know if an action applies in a state? Only if the preconditions successfully unify with the current state. That seems easy enough...you check each predicate in the conditions to see if it unifies with the current state and if it does, you use the substitution list on the action, the add and delete lists and create the successor state based on them.

Except for one small problem...there may be more than one way to unify an action with the current state. You must essentially search for all successful unifications of the candidate action and the current state. This is where my question through the semester appliesm, "how would you modify state space search to return all the paths to the goal?"

Unification can be seen as state space search by trying to unify the first precondition with the current state, progressively working your way through the precondition list. If you fail at any point, you may need to backtrack because there might have been another unification of that predicate that would succeed. Similarly, as already mentioned, there may be more than one.

So...by using unification and a properly defined <code>successors</code> function, you should be able to apply graph based search to the problem and return a "path" through the states from the initial state to the goal. You'll definitely want to use graph-based search since <code>( drive Me Store), (drive Me Home), (drive Me Store), (drive Me Home), (drive Me Store), (buy Me Store Drill), (drive Me Home)</code> is a valid plan.

Your function should return the plan...a list of actions, fully instantiated, for the agent to do in order: [a1, a2, a3]. If you pass an extra intermediate=True parameter, it should also return the resulting state of each action: [s0, a1, s1, a2, s2, a3, s3].

-----

(you can just overwrite that one and add as many others as you need). Remember to follow the **Guidelines**.


-----

So you need to implement `forward_planner` as described above. `start_state`, `goal` and `actions` should all have the layout above and be s-expressions.

Your implementation should return the plan as a **List of instantiated actions**. If `debug=True`, you should print out the intermediate states of the plan as well.

## expression_to_string <a id="expression_to_string"></a>
**Description:**
This function converts a nested expression into a formatted string in the form of an S-expression. It goes through each part of the expression step-by-step, keeping the nested data in the right structure. This is important so that we can understand the problem without needing another format and so other functions can use the string version of the expression.

**Parameters:**
- `expr` (`Any`): The input expression to be converted to string

**Returns:**
- A string formatted as an S-expression.

In [2]:
def expression_to_string(expr: Any) -> str:
    return '(' + ' '.join(map(lambda e: e if isinstance(e, str) else expression_to_string(e), expr)) + ')'

In [3]:
assert expression_to_string(["move", "agent", "location"]) == "(move agent location)"  # Simple expression with a single level

assert expression_to_string(["buy", "agent", ["item", "drill"]]) == "(buy agent (item drill))"  # Nested expression with multiple levels

assert expression_to_string(["drive", ["agent", "me"], ["from", "home"], ["to", "store"]]) == "(drive (agent me) (from home) (to store))"  # Complex expression with varied nesting

## `initialize_start` <a id="initialize_start"></a>

**Description:**  
This function initializes the start state, actions, and goal by parsing each S-expression string into a structured format format that’s easier to use in the rest of the code. It converts the `start_state` and `goal` into lists of parsed expressions and transforms `actions` into a dictionary. Finally, it returns the parsed start state, actions, and a set of goal expressions as strings, making it easier to compare them during the search. This setup helps keep the code simpler, as handling some functions with just plain lists can be frustrating.

**Parameters:**  
- `start_state` (`List[str]`): A list of S-expression strings representing the initial state.
- `actions` (`Dict[str, Dict[str, Any]]`): A dictionary of actions, where each action has conditions, add, and delete lists specified as S-expressions.
- `goal` (`List[str]`): A list of S-expression strings representing the goal state.

**Returns:**  
- A tuple with three elements:
  - `start_parsed` (`List[Any]`): The parsed initial state as structured expressions.
  - `actions_parsed` (`Dict[str, Dict[str, Any]]`): The parsed actions dictionary.
  - `set_goal` (`Set[str]`): A set of goal expressions in string format for easier comparison.

In [4]:
def initialize_start(start_state: List[str], actions: Dict[str, Dict[str, Any]], goal: List[str]) -> Tuple[List[Any], Dict[str, Dict[str, Any]], Set[str]]:
    start_parsed: List[Any] = [parse(singular_state) for singular_state in start_state]
    actions_parsed:  Dict[str, Dict[str, Any]] = {action_name: { key: [parse(expression) for expression in value] if isinstance(value, list) else parse(value)
                for key, value in action_details.items()
            }
            for action_name, action_details in actions.items()
        }
    goal_parsed: List[Any] =  [parse(expression) for expression in goal]
    set_goal: Set[str] = set(expression_to_string(stmt) for stmt in goal_parsed)

    return start_parsed, actions_parsed, set_goal

In [5]:
start_state1 = ["(agent Me)", "(at Me Home)"]
actions1 = {
    "move": {
        "action": "(move ?agent ?from ?to)",
        "conditions": ["(agent ?agent)", "(at ?agent ?from)"],
        "add": ["(at ?agent ?to)"],
        "delete": ["(at ?agent ?from)"]
    }
}
goal1 = ["(at Me Store)", "(agent Me)"]

start_parsed, actions_parsed, set_goal = initialize_start(start_state1, actions1, goal1)
assert (start_parsed == [['agent', 'Me'], ['at', 'Me', 'Home']])  # Should parse each start state expression
assert ("move" in actions_parsed and "action" in actions_parsed["move"])  # Should parse each action into a callable dictionary
assert (set_goal == {'(at Me Store)', '(agent Me)'}) # Goal correctly parsed to set

## `substitute` <a id="substitute"></a>

**Description:**  
This function applies a substitution dictionary to an expression, replacing variables in the expression with values from the dictionary. If the expression is a list (meaning it’s a structured or nested S-expression), the function applies the substitution to each part of the list one by one. This function is key for setting specific values for variables in an action’s conditions, add lists, and delete lists in the forward planner.

**Parameters:**  
- `expression` (`Any`): The expression to which substitutions should be applied. It can be a single variable, a nested list, or an S-expression.
- `sub` (`Dict[str, Any]`): A dictionary containing variable substitutions, where each key is a variable to be replaced and the value is the replacement.

**Returns:**  
- The expression with variables replaced according to the substitution dictionary.

In [6]:
def substitute(expression: Any, sub: Dict[str, Any]) -> Any:
    return [sub.get(e, e) if isinstance(e, str) else substitute(e, sub) for e in expression] if isinstance(expression, list) else sub.get(expression, expression)

In [7]:
expression1 = ["move", "?agent", "?from", "?to"]
sub1 = {"?agent": "Me", "?from": "Home", "?to": "Store"}
assert substitute(expression1, sub1) == ["move", "Me", "Home", "Store"]  # Simple Substitution

expression2 = ["drive", ["agent", "?agent"], ["from", "?from"], ["to", "?to"]]
sub2 = {"?agent": "Me", "?from": "Home", "?to": "Store"}
assert substitute(expression2, sub2) == ["drive", ["agent", "Me"], ["from", "Home"], ["to", "Store"]]  # Substitution in nested lists

expression3 = ["pickup", "?agent", "?item"]
sub3 = {"?other": "Unknown"} 
assert substitute(expression3, sub3) == ["pickup", "?agent", "?item"]  # No substitution should happen

## `generate_new_state` <a id="generate_new_state"></a>

**Description:**  
This function generates a new state by applying an action's `add` and `delete` effects to the current state. It first applies the substitutions to both the `add` and `delete` lists of the action. Then, it removes any expressions in the `delete` list from the current state and adds the expressions in the `add` list to form the new state. This function is essential for updating the state after applying an action within the forward planner.

**Parameters:**  
- `current_states` (`List[Any]`): The current state represented as a list of expressions.
- `action_data` (`Dict[str, List[Any]]`): A dictionary containing the action's `add` and `delete` lists, which specify what expressions to add to and remove from the state.
- `subs` (`Dict[str, Any]`): A dictionary of substitutions, mapping variables to specific values.

**Returns:**  
- A list representing the new state after applying the action's `add` and `delete` effects.

In [8]:
def generate_new_state(current_states, action_data, subs):
    added = list(map(lambda add: substitute(add, subs), action_data['add']))
    removed = list(map(lambda rem: substitute(rem, subs), action_data['delete']))
    return list(filter(lambda state: state not in removed, current_states)) + added

In [9]:
current_state1 = ["(at Me Home)", "(has Money)"]
action_data1 = {
    "add": ["(at Me Store)"],
    "delete": ["(at Me Home)"]
}
subs1 = {}
assert generate_new_state(current_state1, action_data1, subs1) == ["(has Money)", "(at Me Store)"]  # Home removed, Store added

current_state2 = ["(at Me Home)", "(item Drill)", "(at Drill Store)"]
action_data2 = {
    "add": ["(at Drill Me)"],
    "delete": ["(at Drill Store)"]
}
subs2 = {"Drill": "Drill"}
assert generate_new_state(current_state2, action_data2, subs2) == ["(at Me Home)", "(item Drill)", "(at Drill Me)"]  # Store removed, Me added

current_state3 = ["(at Me Home)", "(has Money)"]
action_data3 = {
    "add": [],
    "delete": []
}
subs3 = {}
assert generate_new_state(current_state3, action_data3, subs3) == ["(at Me Home)", "(has Money)"]  # No changes if add and delete lists are empty, also doesnt crash

## `find_applicable_actions` <a id="find_applicable_actions"></a>

**Description:**  
This function identifies all possible substitutions that make an action's conditions applicable to the current state. It attempts to unify each condition with elements in the current state, the generates all valid states. If the conditions unify successfully with the state elements, the function returns a list of possible substitutions that would satisfy the action's preconditions. This is crucial in the forward planner to determine which actions can be applied in a given state.

**Parameters:**  
- `conditions` (`List[Any]`): A list of conditions for an action, represented as expressions that must be unified with elements in the current state.
- `current_state` (`List[Any]`): The current state represented as a list of expressions, used to check if the conditions of an action can be met.

**Returns:**  
- A list of dictionaries, where each dictionary contains substitutions that unify the action’s conditions with the current state.

In [10]:
def find_applicable_actions(conditions: List[Any], current_state: List[Any]) -> List[Dict[str, Any]]:
    possible_actions: List[Dict[str, Any]] = []
    
    unification_attempts: List[Dict[str, Any]] = [{"substitutions": {}, "remaining_conditions": deepcopy(conditions)}]
    while unification_attempts:
        stack_entry = unification_attempts.pop()
        subs = stack_entry["substitutions"]
        conds_left = stack_entry["remaining_conditions"]
        
        for state_elem in current_state:
            temp_subs = unification(conds_left[0], state_elem)
            if temp_subs is not False:
                reverse_existing = {v: k for k, v in subs.items()}
                
                if not any(reverse_existing.get(v) != k for k, v in temp_subs.items() if v in reverse_existing):
                    temp_subs.update(subs)
                    
                    if len(conds_left) == 1:
                        possible_actions.append(temp_subs)
                    else:
                        unification_attempts.append({"substitutions": temp_subs, "remaining_conditions": [substitute(c, temp_subs) for c in conds_left[1:]]})
    
    return possible_actions

In [11]:
conditions1 = [['agent', '?agent'], ['place', '?from'], ['place', '?to'], ['at', '?agent', '?from']]
current_state1 = [['at', 'Me', 'Home'], ['agent', 'Me'], ['place', 'Store'], ['place', 'Home']]
assert find_applicable_actions(conditions1, current_state1) == [{'?to': 'Store', '?from': 'Home', '?agent': 'Me'}]  #Should set to Store from Home

conditions2 = [['agent', '?agent'], ['place', '?from'], ['place', '?to'], ['at', '?agent', '?from']]
current_state2 = [['at', 'Me', 'Home'], ['agent', 'Me'], ['place', 'Store'], ['place', 'Home'], ['place', 'Bank']]
result2 = find_applicable_actions(conditions2, current_state2)
expected_result2 = [{'?to': 'Bank', '?from': 'Home', '?agent': 'Me'}, {'?to': 'Store', '?from': 'Home', '?agent': 'Me'}] #Should also be able to go to Bank
assert result2 == expected_result2  

conditions3 = ["(at ?agent Park)"]
current_state3 = ["(at Me Home)", "(at You Store)"]
assert find_applicable_actions(conditions3, current_state3) == []  # Conditions not met, cant do any moves


## `successors` <a id="successors"></a>

**Description:**  
This function generates successor states by applying all applicable actions to the current state. For each action, it finds valid substitutions that satisfy the action’s preconditions using `find_applicable_actions`. Then, it generates the new state after applying each action’s `add` and `delete` effects. If the resulting state is not in the `explored` set, it is added to the list of successors. This is needed as it returns the new state, the substitutions used, and the action statement, allowing the forward planner to explore new possible states.

**Parameters:**  
- `state` (`List[Any]`): The current state represented as a list of expressions.
- `actions` (`Dict[str, Dict[str, Any]]`): A dictionary of actions, where each action has conditions, add, and delete lists.
- `explored` (`Set[Tuple[str, ...]]`): A set of previously visited states to avoid revisiting and looping.

**Returns:**  
- A list of tuples, where each tuple contains:
  - `new_state` (`List[Any]`): The state resulting from applying an action.
  - `sub` (`Dict[str, Any]`): The substitutions used to apply the action.
  - `action_statement` (`Any`): The fully instantiated action statement.

In [12]:
def successors(state, actions, explored):
    successors_list = []
    for action_name, action_details in actions.items():
        substitutions = find_applicable_actions(action_details['conditions'], state)
        if substitutions:
            for sub in substitutions:
                new_state = generate_new_state(state, action_details, sub)
                action_statement = substitute(action_details['action'], sub)
                new_state_tuple = tuple(expression_to_string(stmt) for stmt in new_state)
                if new_state_tuple not in explored:
                    successors_list.append((new_state, sub, action_statement))
    return successors_list

In [13]:
state1 = ['at', 'Me', 'Home']
actions1 = {
    'drive': {
        'action': ['drive', '?agent', '?from', '?to'],
        'conditions': [['agent', '?agent'], ['place', '?from'], ['place', '?to'], ['at', '?agent', '?from']], 
        'add': [['at', '?agent', '?to']], 
        'delete': [['at', '?agent', '?from']]
        }
}
explored1 = set()
result1 = successors(state1, actions1, explored1)
assert len(result1) == 0  # No successors possible as no destination specified in the state

state2 = [['at', 'Me', 'Home'], ['item', 'Drill'], ['place', 'Bank'],  ['agent', 'Me'],  ['place', 'Home']]
actions2 = {
    'drive': {
        'action': ['drive', '?agent', '?from', '?to'],
        'conditions': [['agent', '?agent'], ['place', '?from'], ['place', '?to'], ['at', '?agent', '?from']], 
        'add': [['at', '?agent', '?to']], 
        'delete': [['at', '?agent', '?from']]
    }
}
explored2 = set()
result2 = successors(state2, actions2, explored2)
assert result2[0][1]["?to"] == "Bank"  # Substitution for destination

state3 = [['at', 'Me', 'Home'], ['item', 'Drill'], ['place', 'Bank'],  ['agent', 'Me'],  ['place', 'Home']]
actions3 = {
    'drive': {
        'action': ['drive', '?agent', '?from', '?to'],
        'conditions': [['agent', '?agent'], ['place', '?from'], ['place', '?to'], ['at', '?agent', '?from']], 
        'add': [['at', '?agent', '?to']], 
        'delete': [['at', '?agent', '?from']]
    }
}
explored3 = [('(item Drill)', '(place Bank)', '(agent Me)', '(place Home)', '(at Me Bank)'), ('(this shouldnt matter)')]
assert successors(state3, actions3, explored3) == []  # No successors, state already explored


## `forward_planner` <a id="forward_planner"></a>

**Description:**  
This function implements a forward planner. It initializes the `start_state`, `actions`, and `goal_set` by parsing them. During each iteration, the function generates successor states and checks if the goal state has been reached. If the goal is achieved, it returns the sequence of actions required to reach the goal. If `debug=True`, the function also includes intermediate states in the output for better traceability. If no solution is found, it returns an empty list.

**Parameters:**  
- `start_state` (`List[str]`): A list of S-expression strings representing the initial state.
- `goal` (`List[str]`): A list of S-expression strings representing the desired goal state.
- `actions` (`Dict[str, Dict[str, Any]]`): A dictionary where each action includes conditions, add, and delete lists.
- `debug` (`bool`): Optional; if set to `True`, the output will include the state at each action step.

**Returns:**  
- A list of action strings required to achieve the goal from the start state, or an empty list if no solution is found. If `debug=True`, intermediate states are also included in the output.

In [14]:
def forward_planner(start_state, goal, actions, debug=False):
    start_state, actions, goal_set = initialize_start(start_state, actions, goal)
    
    explored: Set[Tuple[str, ...]] = set()
    stack: List[Tuple[List[List[Any]], List[Any]]] = [([start_state], [])]    
    plan_output = []

    while stack:
        path, plan = stack.pop()
        current_state = path[-1] 
        
        if goal_set == {expression_to_string(stmt) for stmt in current_state}:
            for i, action in enumerate(plan):
                if debug:
                    plan_output.append(f"{{'state': {[expression_to_string(st) for st in path[i]]}, 'action': {expression_to_string(action)}}}")
                else:
                    plan_output.append(f"{{'action': {expression_to_string(action)}}}")
            plan_output.append(f"{{'final_state': {[expression_to_string(st) for st in path[-1]]}}}")
            return plan_output
        
        current_state_tuple = tuple(expression_to_string(stmt) for stmt in current_state)
        explored.add(current_state_tuple)
        
        successors_list = successors(current_state, actions, explored)
        
        for new_state, sub, action_text in successors_list:
            stack.append((path + [new_state], plan + [action_text]))
    
    return []  # No solution found

In [15]:
start_state1 = ["(at Me Home)", "(agent Me)", "(place Store)", "(place Home)"]
goal1 = ["(at Me Store)", "(agent Me)", "(place Store)", "(place Home)"]
actions1 = {
    "drive": {
        "action": "(drive ?agent ?from ?to)",
        "conditions": ["(at ?agent ?from)", "(agent ?agent)", "(place ?to)"],
        "add": ["(at ?agent ?to)"],
        "delete": ["(at ?agent ?from)"]
    }
}
assert forward_planner(start_state1, goal1, actions1) == ["{'action': (drive Me Home Store)}", \
                                                           "{'final_state': ['(agent Me)', '(place Store)', '(place Home)', '(at Me Store)']}"] # Very Simple Solution

start_state2 = ["(at Me Home)", "(agent Me)", "(place Store)", "(place Home)",  "(item Drill)", "(at Drill Store)"]
goal2 = ["(at Me Home)", "(agent Me)", "(place Store)", "(place Home)", "(item Drill)", "(at Drill Me)"]
actions2 = {
    "drive": {
        "action": "(drive ?agent ?from ?to)",
        "conditions": ["(agent ?agent)", "(at ?agent ?from)", "(place ?to)"],
        "add": ["(at ?agent ?to)"],
        "delete": ["(at ?agent ?from)"]
    },
    "buy": {
        "action": "(buy ?purchaser ?seller ?item)",
        "conditions": ["(at ?item ?seller)", "(at ?purchaser ?seller)", "(agent ?purchaser)", "(item ?item)"],
        "add": ["(at ?item ?purchaser)"],
        "delete": ["(at ?item ?seller)"]
    }
}
result2 = forward_planner(start_state2, goal2, actions2)
assert result2 == ["{'action': (drive Me Home Store)}", \
     "{'action': (buy Me Store Drill)}", \
     "{'action': (drive Me Store Home)}", \
     "{'final_state': ['(agent Me)', '(place Store)', '(place Home)', '(item Drill)', '(at Drill Me)', '(at Me Home)']}"] # Multiple steps to solution

start_state3 = ["(at Me Home)", "(agent Me)", "(place Store)", "(place Home)",  "(item Drill)", "(at Drill Store)"]
goal3 = ["(at Me Home)", "(agent Me)", "(place Store)", "(place Home)", "(item Drill)", "(at Drill Me)"]
actions3 = {
    "drive": {
        "action": "(drive ?agent ?from ?to)",
        "conditions": ["(at ?agent ?from)", "(place ?to)"],
        "add": ["(at ?agent ?to)"],
        "delete": ["(at ?agent ?from)"]
    }
}
assert forward_planner(start_state3, goal3, actions3) == []  # Goal unreachable as no way to purchase, should return []

You will be solving the problem from above. Here is the start state:

In [16]:
start_state = [
    "(item Saw)",
    "(item Drill)",
    "(place Home)",
    "(place Store)",
    "(place Bank)",
    "(agent Me)",
    "(at Me Home)",
    "(at Saw Store)",
    "(at Drill Store)"
]

The goal state:

In [17]:
goal = [
    "(item Saw)",    
    "(item Drill)",
    "(place Home)",
    "(place Store)",
    "(place Bank)",    
    "(agent Me)",
    "(at Me Home)",
    "(at Drill Me)",
    "(at Saw Store)"    
]

and the actions/operators:

In [18]:
actions = {
    "drive": {
        "action": "(drive ?agent ?from ?to)",
        "conditions": [
            "(agent ?agent)",
            "(place ?from)",
            "(place ?to)",
            "(at ?agent ?from)"
        ],
        "add": [
            "(at ?agent ?to)"
        ],
        "delete": [
            "(at ?agent ?from)"
        ]
    },
    "buy": {
        "action": "(buy ?purchaser ?seller ?item)",
        "conditions": [
            "(item ?item)",
            "(place ?seller)",
            "(agent ?purchaser)",
            "(at ?item ?seller)",
            "(at ?purchaser ?seller)"
        ],
        "add": [
            "(at ?item ?purchaser)"
        ],
        "delete": [
            "(at ?item ?seller)"
        ]
    }
}

**Note** The facts for each state are really an ordered set. When comparing two states, you may need to convert them to a Set first.

In [19]:
plan = forward_planner( start_state, goal, actions)

In [20]:
for el in plan:
    print(el)

{'action': (drive Me Home Store)}
{'action': (buy Me Store Drill)}
{'action': (drive Me Store Home)}
{'final_state': ['(item Saw)', '(item Drill)', '(place Home)', '(place Store)', '(place Bank)', '(agent Me)', '(at Saw Store)', '(at Drill Me)', '(at Me Home)']}


## Before You Submit...

1. Did you provide output exactly as requested?
2. Did you re-execute the entire notebook? ("Restart Kernel and Rull All Cells...")
3. If you did not complete the assignment or had difficulty please explain what gave you the most difficulty in the Markdown cell below.
4. Did you change the name of the file to `jhed_id.ipynb`?

Do not submit any other files.